In [9]:
import numpy as np
import cv2
import os
import uuid

In [10]:
def draw_text(img, label, set_label, internel_idx, frame_idx, frame_count):
    height = img.shape[0]
    width = img.shape[1]
    text = ["none", "normal", "paging", "shake", "else"]
    speed_text = ["1X", "0.5X", "0.25X"]
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, str(frame_idx) + '/' + str(frame_count), (10, height - 20), font, 1, (0,0,255), 2)
    cv2.putText(img, "Current set: " + text[set_label + 1], (width - 400, 30), font, 1, (0, 0, 255), 2)
    cv2.putText(img, speed_text[internel_idx], (10, 30), font, 1, (0, 0, 255), 2)
    if label != -1:
        cv2.putText(img, text[label + 1], (width - 200, height - 20), font, 1, (0, 0, 255), 2)

def handle_video(video_path, data_path, data_length, save_width, save_height):

    cap = cv2.VideoCapture(video_path)

    frame_count = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT))
    label_result = [-1] * frame_count
    fps = cap.get(cv2.cv.CV_CAP_PROP_FPS)

    internal = 1000 / fps
    internal_ary = [int(internal), int(internal * 2), int(internal * 4)]
    internal_idx = 0
    frame_idx = 0
    frame_num = 200

    current_label = -1
    
    while cap.isOpened():
        ret, frame = cap.read()
        if ret != True:
            confirm = False
            while True:
                k = cv2.waitKey()
                if k & 0xFF == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    return False
                if k & 0xFF == ord('o'):
                    cap.release()
                    cv2.destroyAllWindows()
                    confirm = True
                    break
                if k & 0xFF == ord('c'):
                    frame_idx = 0
                    cap.set(1, frame_idx)
                    break
            if confirm == True:
                # user presses 'o' to confirm
                break
            else:
                # user presses 'c' to repeat
                continue
        frame_idx = frame_idx + 1
        draw_text(frame, label_result[frame_idx - 1], current_label, internal_idx, frame_idx, frame_count)
        cv2.imshow('frame',frame)
        if current_label != -1:
            label_result[frame_idx - 1] = current_label
        k = cv2.waitKey(internal_ary[internal_idx])
        if k & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            return False
        if k & 0xFF == ord('o'):
            cap.release()
            cv2.destroyAllWindows()
            break
        if k & 0xFF == ord('t'):
            r = cv2.waitKey()
            while r & 0xFF != ord('t'):
                if r & 0xFF == ord('h'):
                    # go back
                    frame_idx = np.max([0, frame_idx - frame_num])
                    cap.set(1, frame_idx)
                    frame_idx = frame_idx + 1
                    ret, frame = cap.read()
                    current_label = -1
                    draw_text(frame, label_result[frame_idx - 1], current_label, internal_idx, frame_idx, frame_count)
                    cv2.imshow('frame', frame)
                if r & 0xFF == ord('l'):
                    # go forward
                    temp = frame_idx
                    frame_idx = np.min([frame_count, frame_idx + frame_num])
                    cap.set(1, frame_idx)
                    frame_idx = frame_idx + 1
                    ret, frame = cap.read()
                    draw_text(frame, label_result[frame_idx - 1], current_label, internal_idx, frame_idx, frame_count)
                    cv2.imshow('frame', frame)
                    for x in range(temp, frame_idx):
                        label_result[x] = current_label
                r = cv2.waitKey()
        if k & 0xFF == ord('h'):
            # go back
            frame_idx = np.max([0, frame_idx - frame_num])
            cap.set(1, frame_idx)
            frame_idx = frame_idx + 1
            ret, frame = cap.read()
            cv2.imshow('frame', frame)
            current_label = -1
        if k & 0xFF == ord('l'):
            # go forward
            temp = frame_idx
            frame_idx = np.min([frame_count - 1, frame_idx + frame_num])
            cap.set(1, frame_idx)
            frame_idx = frame_idx + 1
            ret, frame = cap.read()
            cv2.imshow('frame', frame)
            for x in range(temp, frame_idx):
                label_result[x] = current_label
        if k & 0xFF == ord('j'):
            internal_idx = np.max([internal_idx - 1, 0])
        if k & 0xFF == ord('k'):
            internal_idx = np.min([internal_idx + 1, 2])
        if k & 0xFF == ord('0'):
            current_label = 0
        if k & 0xFF == ord('1'):
            current_label = 1
        if k & 0xFF == ord('2'):
            current_label = 2
        if k & 0xFF == ord('3'):
            current_label = 3
        if k & 0xFF == ord('e'):
            current_label = -1

    cap.release()
    cv2.destroyAllWindows()

    # finally, save the label_result
    cap = cv2.VideoCapture(video_path)
    label_idx = 0
    while True:
        frame_idx = 0
        name = data_path + str(uuid.uuid1()) + '.avi'
        fourcc = cv2.cv.CV_FOURCC(*'XVID')
        out = cv2.VideoWriter(name, fourcc, 25, (save_width, save_height))
        label_start_idx = label_idx
        while cap.isOpened():
            ret, frame = cap.read()
            if ret != True:
                break
            frame = cv2.resize(frame, (save_width, save_height), interpolation = cv2.INTER_AREA)
            out.write(frame)
            frame_idx = frame_idx + 1
            label_idx = label_idx + 1
            if frame_idx >= data_length:
                break
        out.release()
        cv2.destroyAllWindows()
        print(name)
        label_content =label_result[label_start_idx:label_idx]
        label_str = ''.join(map(lambda x: str(x + 1), label_content))
        text_file = open(name.replace('avi', 'txt'), "w")
        text_file.write(label_str)
        text_file.close()
        if frame_idx == 0:
            os.remove(data_path)
        if frame_idx < data_length:
            break
        
    cap.release()
    cv2.destroyAllWindows()
    return 

In [11]:
train_video_path = 'videos/train/'
train_data_path = 'datasets/train/'
data_length = 1000
extension = 'mp4'
save_width = 640
save_height = 480

for (dirpath, dirnames, filenames) in os.walk(train_video_path):
    for filename in filenames:
        if filename.endswith(extension) == False:
            continue
        retval = handle_video(train_video_path + filename, train_data_path, data_length, save_width, save_height)
        if retval == False:
            break

OOOOOOOOOOO
datasets/train/933b0c92-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/9529b47c-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/973414ba-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/9943a70c-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/9b4d0e94-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/9d7d9e4a-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/9f8b2036-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/a18e3b52-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/a38edf10-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/a595196e-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/a79dd674-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/a9b8dd1e-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/abd8a48a-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/adf50e16-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/b014f2ce-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/b22e597e-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/b4426174-a5c3-11e6-9f1b-d017c2cf90d9.avi
datasets/train/b643c260-a5c3-11e6-9f